In [1]:
import os,sys
os.environ['USE_PYGEOS'] = '0'
import rasterio
import shapely
import xarray as xr
import numpy as np
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
from pathlib import Path,PurePath
from typing import Union

import osm_flex.download as dl
import osm_flex.extract as ex

sys.path.append("../src/damagescanner")

data_path = Path('..')

from raster import RasterScanner
from vector import VectorScanner

In [2]:
class DamageScanner(object):
    """DamageScanner - a directe damage assessment toolkit	
    """
    def prepare(self, assessment_type, data_path, curves = pd.DataFrame(), maxdam = dict(),osm_pbf=False,**kwargs):
        """Prepare the input for a damage assessment
        """

        self.assessment_type = assessment_type
        
        self.data_path = data_path       

        # Collect exposure data
        try:
            if self.assessment_type == 'raster':
                self.exposure_data =  list(p.resolve() for p in Path(data_path / 'exposure').glob("**/*") if p.suffix in {".tif", ".tiff", ".nc"})[0]
            elif self.assessment_type == 'vector':
                get_data = list(p.resolve() for p in Path(data_path / 'exposure').glob("**/*") if p.suffix in {".gpkg", ".shp", ".geoparquet",".feather",".pbf"})
                if osm_pbf:
                    self.exposure_data =  list(p.resolve() for p in Path(data_path / 'exposure').glob("**/*") if p.suffix in {".pbf"})[0]
                    self.osm_pbf = osm_pbf
                elif len(get_data) > 0:
                    self.exposure_data =  [p for p in get_data if p.suffix in {".gpkg", ".shp", ".geoparquet",".feather"}][0]
                    self.osm_pbf = False
                elif len([p for p in get_data if p.suffix in {".gpkg", ".shp", ".geoparquet",".feather"}]) == 0:
                    self.exposure_data =  list(p.resolve() for p in Path(data_path / 'exposure').glob("**/*") if p.suffix in {".pbf"})[0]     
                    self.osm_pbf = osm_pbf
        except:
            raise ImportError(": No exposure data found. Either use the download() function to download OpenStreetMap data, or manually add data to the exposure folder")
        
        # Collect hazard data
        self.hazard_data = list(p.resolve() for p in Path(data_path / 'hazard').glob("**/*") if p.suffix in {".tif", ".tiff", ".nc"})

        if len(self.hazard_data) == 0:
            raise ImportError("NOTE: No hazard data found. Add data to the hazard folder first")

        # Collect vulnerability curves
        if len(curves) > 0:
            self.curves = curves
        else:
            self.curves = list(p.resolve() for p in Path(data_path / 'vulnerability').glob("**/*") if 'curve' in str(p).lower())[0]

        # Collect maxdam information
        if len(maxdam) > 0:
            self.maxdam = maxdam
        else:
            self.maxdam = list(p.resolve() for p in Path(data_path / 'vulnerability').glob("**/*") if (('dam' in str(p).lower()) |
                                                                                                       ('cost' in str(p).lower()))) [0]
       
        
    def single_event(self,hazard_type='',hazard_file='',return_period='',save_output=False,vector_landuse=True,**kwargs):
        """Damage assessment for a single event. Can be a specific hazard event, or a specific single hazard footprint 
        """

        if not hasattr(self, 'assessment_type'):
            raise ImportError('ERROR: Please run .prepare() first to set up the assessment')

        if hazard_file != '':
            self.hazard_map = [x for x in self.hazard_data if hazard_file in str(x)][0]
        else:      
            if hazard_type == '':
                if len(self.hazard_data) == 1:
                    self.hazard_map = self.hazard_data[0]
                else:
                    raise RuntimeError('Multiple events require the .multiple_events() function!')     
            elif hazard_type != '':
                if hazard_type == 'flood':
                    hazard_characteristics = ['inun','flood','fluvial','pluvial','coastal','fd','pd']
                elif hazard_type == 'windstorm':
                    hazard_characteristics = ['storm','wind','cyclone','tc']
                elif hazard_type == 'earthquake':
                    hazard_characteristics = ['eq','earthquake']
                elif hazard_type == 'landslide':
                    hazard_characteristics = ['ls','landslide']
                
                hazard_maps = [x for x in self.hazard_data if any(y in str(x).lower() for y in hazard_characteristics)]
                
                if return_period != '':
                    self.hazard_map = [x for x in hazard_maps if return_period in str(x)].lower()[0]
                
                elif return_period == '':
                    self.hazard_map = hazard_maps[0]
                
                else:
                    raise RuntimeError('Specify filename or flood type and return period to be able to run a single event analysis. Multiple events require the .multiple_events() function.')           
            
        if self.assessment_type == 'raster':

            return RasterScanner(
                            exposure_file = self.exposure_data,
                            hazard_file = self.hazard_map,
                            curve_path = self.curves,
                            maxdam_path = self.maxdam,
                            save=save_output)

        elif self.assessment_type == 'vector':

            # specificy essential data input characteristics
            if 'cell_size' in kwargs:
                self.cell_size = kwargs.get('cell_size')#0.01666,
            else:
                self.cell_size = 5
    
            if 'exp_crs' in kwargs:
                self.exp_crs = kwargs.get('exp_crs')
            else:
                self.exp_crs = 4326
    
            if 'haz_crs' in kwargs:
                self.haz_crs = kwargs.get('haz_crs')
            else:
                self.haz_crs = 4326
    
            if 'object_col' in kwargs:
                self.object_col = kwargs.get('object_col')
            else:
                self.object_col = 'landuse'
    
            if 'hazard_col' in kwargs:
                self.hazard_col = kwargs.get('hazard_col')
            else:
                self.hazard_col = 'inun_val'
    
            if 'lat_col' in kwargs:
                self.lat_col = kwargs.get('lat_col')
            else:
                self.lat_col = 'y'
    
            if 'lon_col' in kwargs:
                self.lon_col = kwargs.get('lon_col')
            else:
                self.lon_col = 'x'
    
            if 'centimers' in kwargs:
                self.centimers = kwargs.get('centimers')#0.01666,#5,
            else:
                self.centimers = False
            
            if self.osm_pbf:
                if 'buildings' in kwargs:
                    self.exposure_data = DamageScanner.buildings(self)                  
                    self.exposure_data = self.exposure_data.rename({'building':'element_type'},axis=1)       
                    
                elif 'roads' in kwargs:
                    self.exposure_data = DamageScanner.cis(self,infra_type='road')
                    self.exposure_data = self.exposure_data.rename({'highway':'element_type'},axis=1)       
                
                elif vector_landuse:
                    self.exposure_data = DamageScanner.landuse(self)
                    self.exposure_data = self.exposure_data.rename({'landuse':'element_type'},axis=1)       

            
            return VectorScanner(
                            exposure_file = self.exposure_data,
                            hazard_file = self.hazard_map,
                            curve_path = self.curves,
                            maxdam_path = self.maxdam,
                            cell_size = self.cell_size, #0.01666,#5,
                            exp_crs = self.exp_crs, #28992,
                            haz_crs = self.haz_crs, #4326,
                            object_col= self.object_col, #'landuse',
                            hazard_col= self.hazard_col,
                            lat_col = self.lat_col,
                            lon_col = self.lon_col,
                            centimeters= self.centimers, #False,
                            save=save_output)  

                
    def multiple_events(self):
        """Damage assessment for multiple events. Also the input for the risk assessment
        """
        pass

    def risk(self):
        """Risk assessment
        """
        pass

    def landuse(self):
        """
        """
        extracted_exposure = ex.extract(
                            	self.exposure_data, 'multipolygons', ['landuse'])    
        
        extracted_exposure.geometry = shapely.make_valid(extracted_exposure.geometry)        

        return extracted_exposure
    
    def buildings(self):
        """
        """
        extracted_exposure = ex.extract(
                            	self.exposure_data, 'multipolygons', ['building'])    
        
        extracted_exposure.geometry = shapely.make_valid(extracted_exposure.geometry)        

        return extracted_exposure

    def cis(self,infra_type):
        """
        """
        extracted_exposure = ex.extract_cis(self.exposure_data, infra_type)

        if infra_type == 'road':
            extracted_exposure = extracted_exposure.loc[extracted_exposure.geometry.geom_type == 'LineString']
        
        extracted_exposure.geometry = shapely.make_valid(extracted_exposure.geometry)        

        return extracted_exposure
        
    def cis_all(self,to_exclude=[]):
        """
        """
        cis = ['healthcare', 'education', 'gas', 'oil', 'telecom', 'water', 'wastewater', 'power', 'rail', 'road', 'air']
        pass

    def download(self, country_code='JAM'):
        """
        """        
        pass

In [3]:
kampen = DamageScanner()

In [4]:
%%time
kampen.prepare(assessment_type = 'raster',
                data_path = data_path / 'data' / 'kampen')

kampen_flood = kampen.single_event(hazard_type = 'flood')

CPU times: total: 1.11 s
Wall time: 1.11 s


In [12]:
maxdam = {"grass":5,
        "forest":10,
        "orchard":50,
        "residential":200,
        "education":200,
        "industrial":300,
        "retail":300,
        "railway":100,
        "farmland":10,
        "cemetery":15,
        "construction":10,
        "meadow":5,
        "farmyard":5,
        "scrub":5,
        "allotments":10,
        "reservoir":5,
        "static_caravan":100,
        "commercial":300,
        'unclassified':100, 
        'primary':100, 
        'secondary':100, 
        'tertiary':100, 
        'residential':100,
        'trunk':100, 
        'trunk_link':100, 
        'motorway':100, 
        'motorway_link':100, 
        'primary_link':100, 
        'secondary_link':100,
        'tertiary_link':100,
        'yes':300,
        'house':300, 
        'apartments':300, 
        'roof':300, 
        'houseboat':300,
        'greenhouse':300, 
        'school':300,
        'church':300, 
        'civic':300, 
        'shed':300, 
        'garage':300, 
        'office':300, 
        'tech_cab':300,
        'fire_station':300, 
        'public':300, 
        'service':300, 
        'factory':300, 
        'farm_auxiliary':300, 
        'storage_tank':300, 
        'parking':300, 
        'bicycle shed':300}

curves = np.array(
            [[0,0],
            [50,0.2],
            [100,0.4],
            [150,0.6],
            [200,0.8],
            [250,1]])  
    
curves = np.concatenate((curves,
                            np.transpose(np.array([curves[:,1]]*(len(maxdam)-1)))),
                           axis=1)

curves = pd.DataFrame(curves)
curves.columns = ['depth']+list(maxdam.keys())
curves.set_index('depth',inplace=True)    

In [17]:
kampen.prepare(assessment_type = 'vector', 
               data_path = data_path / 'data' / 'kampen', 
                curves = curves, 
                maxdam = maxdam,
               osm_pbf=True,
)

In [15]:
%%time
kampen_buildings = kampen.single_event(hazard_type = 'windstorm',
                            buildings=True,
                            exp_crs = 4326,
                            haz_crs = 4326,
                            cell_size = 0.166667,
                            object_col= 'highway',
                            hazard_col= 'FX',
                            lat_col = 'Latitude',
                            lon_col = 'Longitude',
                            centimeters= True )

extract multipolygons: 100%|██████████████████████████████████████████████████| 29673/29673 [00:01<00:00, 23851.68it/s]


PROGRESS: Exposure and hazard data loaded


100%|███████████████████████████████████████████████████████████████████████████| 29673/29673 [00:40<00:00, 740.03it/s]


CPU times: total: 44.3 s
Wall time: 44.3 s


In [ ]:
kampen.prepare(assessment_type = 'vector', 
               data_path = data_path / 'data' / 'kampen', 
                curves = curves, 
                maxdam = maxdam,
               osm_pbf=True,
)

In [18]:
%%time
kampen_roads = kampen.single_event(hazard_type = 'flood',
                            roads=True,
                            exp_crs = 4326,
                            haz_crs = 28992,
                            cell_size = 5,
                            object_col= 'landuse',
                            centimeters= False )

extract points: 0it [00:00, ?it/s]
extract multipolygons: 0it [00:00, ?it/s]
extract lines: 100%|████████████████████████████████████████████████████████████| 2651/2651 [00:00<00:00, 11051.43it/s]


PROGRESS: Exposure and hazard data loaded


100%|█████████████████████████████████████████████████████████████████████████████| 1905/1905 [00:01<00:00, 970.26it/s]


CPU times: total: 53.8 s
Wall time: 53.9 s


In [19]:
jamaica = DamageScanner()
jamaica.prepare(assessment_type = 'vector', 
                data_path = data_path / 'data' / 'jamaica',                 
                curves = curves, 
                maxdam = maxdam,
                osm_pbf=True,
                
)

In [20]:
%%time
jamaica_windstorm = jamaica.single_event(hazard_type = 'flood',
                            roads=True,
                            exp_crs = 4326,
                            haz_crs = 4326,
                            cell_size = 0.1,
                            object_col= 'highway',
                            centimeters= False )

extract points: 0it [00:00, ?it/s]
extract lines: 100%|███████████████████████████████████████████████████████████| 39952/39952 [00:04<00:00, 9462.96it/s]


PROGRESS: Exposure and hazard data loaded


100%|████████████████████████████████████████████████████████████████████████████| 37987/37987 [14:53<00:00, 42.50it/s]


CPU times: total: 16min 32s
Wall time: 16min 31s
